In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import glob
from elasticsearch import Elasticsearch
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
images= glob.glob("images/*.jpg")

In [ ]:
num_classes = 37
IMAGE_SHAPE = (256, 256)

m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_152/feature_vector/4",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
m.build([None, 256, 256, 3])  # Batch input shape.

In [ ]:
image = "images/Abyssinian_1.jpg"

image = Image.open(image)
image = np.asarray(image)
image = np.array(image)/255.0

plt.imshow(image)
plt.axis('off')

print(image.shape)

plt.show()

In [ ]:
result = m.predict(image[np.newaxis, ...])

In [ ]:
es = Elasticsearch("localhost:9200")
def register(name, vector):
    index = "document7"

    document = {
        "filename": name,
        "image_vector": vector
    }

    es.index(index=index, body=document)

In [ ]:
s = []
for image in images:
    name = image
    image = Image.open(image).resize((256, 256)) # 256 * 256の画像に変換
    image = image.convert("RGB")
    image = np.asarray(image)
    if len(image.shape) == 3:
        image = np.array(image)/255.0 # 色を0 - 1 に変換
        vec = m.predict(image[np.newaxis, ...])
        register(name, vec[0]. tolist())